In [2]:
import os
import numpy as np
import tensorflow as tf
import pandas as pd

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import load_model

import cvlib as cv
import cv2
import face_recognition

from PIL import ImageFont, ImageDraw, Image
 

C:\Users\kimsh\Anaconda3\envs\env\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\kimsh\Anaconda3\envs\env\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\kimsh\Anaconda3\envs\env\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\kimsh\Anaconda3\envs\env\lib\site-packages\tensorflow\python\framework

In [3]:
model = load_model('models/model.h5')
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
flatten_1 (Flatten)          (None, 100352)            0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               12845184  
_________________________________________________________________
batch_normalization_v1_1 (Ba (None, 128)               512       
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 129       
Total params: 36,433,537
Trainable params: 36,380,161
Non-trainable params: 53,376
_________________________________________________________________


In [4]:
# 경로 변경
cd ./maskdata/face_recognition/knowns/

C:\Users\kimsh\[K-Digital] MLDL\Semi-project 3 (데이터 수집-전처리-통계분석-시각화 + ML _ DL)_PM\maskdata\face_recognition\knowns


In [5]:
curr_path = os.getcwd()
curr_path

'C:\\Users\\kimsh\\[K-Digital] MLDL\\Semi-project 3 (데이터 수집-전처리-통계분석-시각화 + ML _ DL)_PM\\maskdata\\face_recognition\\knowns'

In [6]:
known_face_dir = os.listdir('./')
known_face_dir

['kimsoohee.jpg']

In [7]:
# 아는 얼굴들 미리 모아놓기

known_face_encodings = []
known_user_names = []
error_user_list = []

for known_face_user in known_face_dir:
    
    known_user_image = face_recognition.load_image_file(known_face_user)
    
    try:
        known_user_encoding = face_recognition.face_encodings(known_user_image)[0]
        known_face_encodings.append(known_user_encoding)
        known_user_names.append(known_face_user[:-4])
        
    except:
        print(f"{known_face_user[:-4]}의 얼굴인식이 되지 않습니다.재촬영이 필요합니다.")
        error_user_list.append(known_face_user)
        
        

In [8]:

def known_face_name(frame, known_face_encodings, known_user_names):
    face_locations = []
    face_encodings = []
    face_names = []
    process_this_frame = True
    name = "Unknown"

    # 얼굴 인식 처리 속도를 높이기 위해 비디오 프레임 크기를 1/4 크기로 조정
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

    # 영상을 BGR 색상(OpenCV가 사용하는)에서 RGB 색상(페이스_인식 사용)으로 변환
    rgb_small_frame = small_frame[:, :, ::-1]

    # 다른 모든 비디오 프레임만 처리하여 시간 절약
    if process_this_frame:
        # 현재 비디오 프레임에서 모든 얼굴 및 얼굴 인코딩 찾기
        face_locations = face_recognition.face_locations(rgb_small_frame)
        face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

        face_names = []
        for face_encoding in face_encodings:
            # 얼굴이 known face(s)와 일치하는지 확인
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            
            # # known_face_encodings에서 일치하는 항목이 발견되면 첫 번째 항목을 사용
            # 일치하는 경우:
            # first_match_index =이(가) 일치합니다.index(참)
            # name = knowled_face_names[first_match_index]

            # 대신 새 얼굴까지의 거리가 가장 작은 알려진 얼굴을 사용합니다.
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_user_names[best_match_index]

        return name
    

In [9]:
video_capture = cv2.VideoCapture(0)
face_locations = []
face_encodings = []
face_names = []
process_this_frame = True

In [17]:
# 상위 폴더로 올라가놓기
cd ../

C:\Users\kimsh\[K-Digital] MLDL\Semi-project 3 (데이터 수집-전처리-통계분석-시각화 + ML _ DL)_PM\maskdata\face_recognition


In [22]:
curr_path = os.getcwd()
curr_path

'C:\\Users\\kimsh\\[K-Digital] MLDL\\Semi-project 3 (데이터 수집-전처리-통계분석-시각화 + ML _ DL)_PM\\maskdata\\face_recognition'

In [23]:
from datetime import datetime

nomask_info = []

# open webcam
webcam = cv2.VideoCapture(0)
 
if not webcam.isOpened():
    print("Could not open webcam")
    exit()
    
 
# loop through frames
while webcam.isOpened():
 
    # read frame from webcam 
    status, frame = webcam.read()
    
    if not status:
        print("Could not read frame")
        exit()
 
    # apply face detection
    face, confidence = cv.detect_face(frame)
 
    # loop through detected faces
    for idx, f in enumerate(face):
        
        (startX, startY) = f[0], f[1]
        (endX, endY) = f[2], f[3]
        
        if 0 <= startX <= frame.shape[1] and 0 <= endX <= frame.shape[1] and 0 <= startY <= frame.shape[0] and 0 <= endY <= frame.shape[0]:
            
            face_region = frame[startY:endY, startX:endX]
            
            face_region1 = cv2.resize(face_region, (224, 224), interpolation = cv2.INTER_AREA)
            
            x = img_to_array(face_region1)
            x = np.expand_dims(x, axis=0)
            x = preprocess_input(x)
            
            prediction = model.predict(x)
 
            if prediction < 0.5: # 마스크 미착용으로 판별되면, 
                result_name = known_face_name(frame, known_face_encodings, known_user_names)
#                 print(result_name)
                cv2.rectangle(frame, (startX,startY), (endX,endY), (0,0,255), 2)
                Y = startY - 10 if startY - 10 > 10 else startY + 10
                text = "No Mask ({:.2f}%)".format((1 - prediction[0][0])*100)
                result_text = result_name + " is " + text
                cv2.putText(frame, result_text, (startX,Y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,0,255), 2)
                
                # 이미지 저장
#                 now_time = datetime.now().strftime('%Y_%m_%d_%H_%M_%S') 
                now_time = datetime.now().strftime('%Y_%m_%d_%H_%M_%S') 
                #초단위라서 여러장 나옴 #이 위에 중복인물인지 체크하는 함수와 if문으로 저장 필요%H%M%S
#                 temp_time = now_time[:17]
                
                # 사용자에 따라서 폴더 만들기
                nomask_capture_path = './nomask_capture/'
                now_time = datetime.now().strftime('%Y_%m_%d_%H_%M_%S') 
                name_path = nomask_capture_path + result_name
#                 print("name_path->>>>", name_path)
                
                if not os.path.isdir(name_path):
                    os.mkdir(name_path)

                img_name = f"{name_path}/{result_name}{now_time}.png".format(name_path, result_name, now_time)
                cv2.imwrite(img_name, frame)    
#                 print(img_name)
                nomask_info.append((result_name, now_time))

#                 os.chdir('../')
                
            else: # 마스크 착용으로 판별되면
                cv2.rectangle(frame, (startX,startY), (endX,endY), (0,255,0), 2)
                Y = startY - 10 if startY - 10 > 10 else startY + 10
                text = "Mask ({:.2f}%)".format(prediction[0][0]*100)
                cv2.putText(frame, text, (startX,Y), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)
                
    # display output
    cv2.imshow("mask nomask classify", frame)
 
    # press "Q" to stop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        df = pd.DataFrame(nomask_info, columns=['name', 'date_time'])
        
        list_path = curr_path + '\\nomask_list'
        print("listPath->>", list_path)
        if not os.path.isdir(list_path):                                                           
            os.mkdir(list_path)
            
        df.to_excel('{}\\nomask_list_{}.xlsx'.format(list_path, datetime.now().strftime('%Y_%m_%d')))
        break
    
# release resources
webcam.release()
cv2.destroyAllWindows() 


name_path->>>> ./nomask_capture/kimsoohee
./nomask_capture/kimsoohee/kimsoohee2021_01_31_15_05_28.png
name_path->>>> ./nomask_capture/kimsoohee
./nomask_capture/kimsoohee/kimsoohee2021_01_31_15_05_29.png
name_path->>>> ./nomask_capture/kimsoohee
./nomask_capture/kimsoohee/kimsoohee2021_01_31_15_05_30.png
name_path->>>> ./nomask_capture/kimsoohee
./nomask_capture/kimsoohee/kimsoohee2021_01_31_15_05_31.png
listPath->> C:\Users\kimsh\[K-Digital] MLDL\Semi-project 3 (데이터 수집-전처리-통계분석-시각화 + ML _ DL)_PM\maskdata\face_recognition\nomask_list


In [52]:
# release resources
video_capture.release()
cv2.destroyAllWindows() 